In [1]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = RandomOuterStrategy(prob_norm="sum")


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

0: ((λ#2. (λ#3. #3)) ((λ#4. (#4 #4)) (λ#5. (#5 #5))))
1: (λ#6. #6)


reduction steps: 1
norm term:       (λ#6. #6)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λ#16. (λ#17. (λ#18. ((#16 #17) #18)))) (λ#19. (λ#20. #19))) (λ#21. (λ#22. (#21 (#21 (#21 #22)))))) (λ#23. (λ#24. (#23 #24))))
1: (((λ#25. (λ#26. (((λ#27. (λ#28. #27)) #25) #26))) (λ#29. (λ#30. (#29 (#29 (#29 #30)))))) (λ#31. (λ#32. (#31 #32))))
2: ((λ#33. (((λ#34. (λ#35. #34)) (λ#36. (λ#37. (#36 (#36 (#36 #37)))))) #33)) (λ#38. (λ#39. (#38 #39))))
3: (((λ#40. (λ#41. #40)) (λ#42. (λ#43. (#42 (#42 (#42 #43)))))) (λ#44. (λ#45. (#44 #45))))
4: ((λ#46. (λ#47. (λ#48. (#47 (#47 (#47 #48)))))) (λ#49. (λ#50. (#49 #50))))
5: (λ#51. (λ#52. (#51 (#51 (#51 #52)))))


reduction steps: 5
norm term:       (λ#51. (λ#52. (#51 (#51 (#51 #52)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λ#62. (λ#63. (λ#64. ((#62 #63) #64)))) (λ#65. (λ#66. #66))) (λ#67. (λ#68. #68))) (λ#69. (λ#70. (#69 (#69 (#69 (#69 #70)))))))
1: (((λ#71. (λ#72. (((λ#73. (λ#74. #74)) #71) #72))) (λ#75. (λ#76. #76))) (λ#77. (λ#78. (#77 (#77 (#77 (#77 #78)))))))
2: ((λ#79. (((λ#80. (λ#81. #81)) (λ#82. (λ#83. #83))) #79)) (λ#84. (λ#85. (#84 (#84 (#84 (#84 #85)))))))
3: (((λ#86. (λ#87. #87)) (λ#88. (λ#89. #89))) (λ#90. (λ#91. (#90 (#90 (#90 (#90 #91)))))))
4: ((λ#92. #92) (λ#93. (λ#94. (#93 (#93 (#93 (#93 #94)))))))
5: (λ#95. (λ#96. (#95 (#95 (#95 (#95 #96))))))


reduction steps: 5
norm term:       (λ#95. (λ#96. (#95 (#95 (#95 (#95 #96))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λ#105. (λ#106. (λ#107. (((#105 (λ#108. (λ#109. (#109 (#108 #106))))) (λ#110. #107)) (λ#111. #111))))) (λ#112. (λ#113. #113)))
1: (λ#114. (λ#115. ((((λ#116. (λ#117. #117)) (λ#118. (λ#119. (#119 (#118 #114))))) (λ#120. #115)) (λ#121. #121))))
2: (λ#122. (λ#123. (((λ#124. #124) (λ#125. #123)) (λ#126. #126))))
3: (λ#127. (λ#128. ((λ#129. #128) (λ#130. #130))))
4: (λ#131. (λ#132. #132))


reduction steps: 4
norm term:       (λ#131. (λ#132. #132))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λ#141. (λ#142. (λ#143. (((#141 (λ#144. (λ#145. (#145 (#144 #142))))) (λ#146. #143)) (λ#147. #147))))) (λ#148. (λ#149. (#148 (#148 (#148 (#148 (#148 #149))))))))
1: (λ#150. (λ#151. ((((λ#152. (λ#153. (#152 (#152 (#152 (#152 (#152 #153))))))) (λ#154. (λ#155. (#155 (#154 #150))))) (λ#156. #151)) (λ#157. #157))))
2: (λ#158. (λ#159. (((λ#160. ((λ#161. (λ#162. (#162 (#161 #158)))) ((λ#163. (λ#164. (#164 (#163 #158)))) ((λ#165. (λ#166. (#166 (#165 #158)))) ((λ#167. (λ#168. (#168 (#167 #158)))) ((λ#169. (λ#170. (#170 (#169 #158)))) #160)))))) (λ#171. #159)) (λ#172. #172))))
3: (λ#173. (λ#174. (((λ#175. (λ#176. (#176 (#175 #173)))) ((λ#177. (λ#178. (#178 (#177 #173)))) ((λ#179. (λ#180. (#180 (#179 #173)))) ((λ#181. (λ#182. (#182 (#181 #173)))) ((λ#183. (λ#184. (#184 (#183 #173)))) (λ#185. #174)))))) (λ#186. #186))))
4: (λ#187. (λ#188. (((λ#189. (λ#190. (#190 (#189 #187)))) (λ#191. (#191 (((λ#192. (λ#193. (#193 (#192 #187)))) ((λ#194. (λ#195. (#195 (#194 #187)))) ((λ#196. (λ#197. (#197 (#19

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λ#286. (λ#287. ((#287 (λ#288. (λ#289. (λ#290. (#289 ((#288 #289) #290)))))) #286))) (λ#291. (λ#292. #292))) (λ#293. (λ#294. #294)))
1: ((λ#295. ((#295 (λ#296. (λ#297. (λ#298. (#297 ((#296 #297) #298)))))) (λ#299. (λ#300. #300)))) (λ#301. (λ#302. #302)))
2: (((λ#303. (λ#304. #304)) (λ#305. (λ#306. (λ#307. (#306 ((#305 #306) #307)))))) (λ#308. (λ#309. #309)))
3: ((λ#310. #310) (λ#311. (λ#312. #312)))
4: (λ#313. (λ#314. #314))


reduction steps: 4
norm term:       (λ#313. (λ#314. #314))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λ#324. (λ#325. ((#325 (λ#326. (λ#327. (λ#328. (#327 ((#326 #327) #328)))))) #324))) (λ#329. (λ#330. (#329 (#329 #330))))) (λ#331. (λ#332. (#331 (#331 (#331 #332))))))
1: ((λ#333. ((#333 (λ#334. (λ#335. (λ#336. (#335 ((#334 #335) #336)))))) (λ#337. (λ#338. (#337 (#337 #338)))))) (λ#339. (λ#340. (#339 (#339 (#339 #340))))))
2: (((λ#341. (λ#342. (#341 (#341 (#341 #342))))) (λ#343. (λ#344. (λ#345. (#344 ((#343 #344) #345)))))) (λ#346. (λ#347. (#346 (#346 #347)))))
3: ((λ#348. ((λ#349. (λ#350. (λ#351. (#350 ((#349 #350) #351))))) ((λ#352. (λ#353. (λ#354. (#353 ((#352 #353) #354))))) ((λ#355. (λ#356. (λ#357. (#356 ((#355 #356) #357))))) #348)))) (λ#358. (λ#359. (#358 (#358 #359)))))
4: ((λ#360. (λ#361. (λ#362. (#361 ((#360 #361) #362))))) ((λ#363. (λ#364. (λ#365. (#364 ((#363 #364) #365))))) ((λ#366. (λ#367. (λ#368. (#367 ((#366 #367) #368))))) (λ#369. (λ#370. (#369 (#369 #370)))))))
5: (λ#371. (λ#372. (#371 ((((λ#373. (λ#374. (λ#375. (#374 ((#373 #374) #375))))) ((λ#376. (λ#377. (λ#37

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λ#437. (λ#438. ((#438 (λ#439. (λ#440. (λ#441. (((#439 (λ#442. (λ#443. (#443 (#442 #440))))) (λ#444. #441)) (λ#445. #445)))))) #437))) (λ#446. (λ#447. (#446 (#446 (#446 (#446 #447))))))) (λ#448. (λ#449. (#448 #449))))
1: ((λ#450. ((#450 (λ#451. (λ#452. (λ#453. (((#451 (λ#454. (λ#455. (#455 (#454 #452))))) (λ#456. #453)) (λ#457. #457)))))) (λ#458. (λ#459. (#458 (#458 (#458 (#458 #459)))))))) (λ#460. (λ#461. (#460 #461))))
2: (((λ#462. (λ#463. (#462 #463))) (λ#464. (λ#465. (λ#466. (((#464 (λ#467. (λ#468. (#468 (#467 #465))))) (λ#469. #466)) (λ#470. #470)))))) (λ#471. (λ#472. (#471 (#471 (#471 (#471 #472)))))))
3: ((λ#473. ((λ#474. (λ#475. (λ#476. (((#474 (λ#477. (λ#478. (#478 (#477 #475))))) (λ#479. #476)) (λ#480. #480))))) #473)) (λ#481. (λ#482. (#481 (#481 (#481 (#481 #482)))))))
4: ((λ#483. (λ#484. (λ#485. (((#483 (λ#486. (λ#487. (#487 (#486 #484))))) (λ#488. #485)) (λ#489. #489))))) (λ#490. (λ#491. (#490 (#490 (#490 (#490 #491)))))))
5: (λ#492. (λ#493. ((((λ#494. (λ#495. (#494 (

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λ#602. (λ#603. ((#603 (λ#604. (λ#605. (λ#606. (((#604 (λ#607. (λ#608. (#608 (#607 #605))))) (λ#609. #606)) (λ#610. #610)))))) #602))) (λ#611. (λ#612. (#611 (#611 (#611 #612)))))) (λ#613. (λ#614. (#613 (#613 (#613 (#613 (#613 #614))))))))
1: ((λ#615. ((#615 (λ#616. (λ#617. (λ#618. (((#616 (λ#619. (λ#620. (#620 (#619 #617))))) (λ#621. #618)) (λ#622. #622)))))) (λ#623. (λ#624. (#623 (#623 (#623 #624))))))) (λ#625. (λ#626. (#625 (#625 (#625 (#625 (#625 #626))))))))
2: (((λ#627. (λ#628. (#627 (#627 (#627 (#627 (#627 #628))))))) (λ#629. (λ#630. (λ#631. (((#629 (λ#632. (λ#633. (#633 (#632 #630))))) (λ#634. #631)) (λ#635. #635)))))) (λ#636. (λ#637. (#636 (#636 (#636 #637))))))
3: ((λ#638. ((λ#639. (λ#640. (λ#641. (((#639 (λ#642. (λ#643. (#643 (#642 #640))))) (λ#644. #641)) (λ#645. #645))))) ((λ#646. (λ#647. (λ#648. (((#646 (λ#649. (λ#650. (#650 (#649 #647))))) (λ#651. #648)) (λ#652. #652))))) ((λ#653. (λ#654. (λ#655. (((#653 (λ#656. (λ#657. (#657 (#656 #654))))) (λ#658. #655)) (λ#659. #6

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λ#1719. (λ#1720. ((λ#1721. ((#1721 (λ#1722. (λ#1723. (λ#1724. #1724)))) (λ#1725. (λ#1726. #1725)))) (((λ#1727. (λ#1728. ((#1728 (λ#1729. (λ#1730. (λ#1731. (((#1729 (λ#1732. (λ#1733. (#1733 (#1732 #1730))))) (λ#1734. #1731)) (λ#1735. #1735)))))) #1727))) #1719) #1720)))) (λ#1736. (λ#1737. (#1736 (#1736 (#1736 #1737)))))) (λ#1738. (λ#1739. (#1738 (#1738 #1739)))))
1: (((λ#1740. (λ#1741. ((λ#1742. ((#1742 (λ#1743. (λ#1744. (λ#1745. #1745)))) (λ#1746. (λ#1747. #1746)))) ((λ#1748. ((#1748 (λ#1749. (λ#1750. (λ#1751. (((#1749 (λ#1752. (λ#1753. (#1753 (#1752 #1750))))) (λ#1754. #1751)) (λ#1755. #1755)))))) #1740)) #1741)))) (λ#1756. (λ#1757. (#1756 (#1756 (#1756 #1757)))))) (λ#1758. (λ#1759. (#1758 (#1758 #1759)))))
2: (((λ#1760. (λ#1761. ((((λ#1762. ((#1762 (λ#1763. (λ#1764. (λ#1765. (((#1763 (λ#1766. (λ#1767. (#1767 (#1766 #1764))))) (λ#1768. #1765)) (λ#1769. #1769)))))) #1760)) #1761) (λ#1770. (λ#1771. (λ#1772. #1772)))) (λ#1773. (λ#1774. #1773))))) (λ#1775. (λ#1776. (#1775 (#1775 (#1

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λ#2289. (λ#2290. ((λ#2291. ((#2291 (λ#2292. (λ#2293. (λ#2294. #2294)))) (λ#2295. (λ#2296. #2295)))) (((λ#2297. (λ#2298. ((#2298 (λ#2299. (λ#2300. (λ#2301. (((#2299 (λ#2302. (λ#2303. (#2303 (#2302 #2300))))) (λ#2304. #2301)) (λ#2305. #2305)))))) #2297))) #2289) #2290)))) (λ#2306. (λ#2307. (#2306 (#2306 #2307))))) (λ#2308. (λ#2309. (#2308 (#2308 (#2308 (#2308 (#2308 #2309))))))))
1: (((λ#2310. (λ#2311. (((((λ#2312. (λ#2313. ((#2313 (λ#2314. (λ#2315. (λ#2316. (((#2314 (λ#2317. (λ#2318. (#2318 (#2317 #2315))))) (λ#2319. #2316)) (λ#2320. #2320)))))) #2312))) #2310) #2311) (λ#2321. (λ#2322. (λ#2323. #2323)))) (λ#2324. (λ#2325. #2324))))) (λ#2326. (λ#2327. (#2326 (#2326 #2327))))) (λ#2328. (λ#2329. (#2328 (#2328 (#2328 (#2328 (#2328 #2329))))))))
2: (((λ#2330. (λ#2331. ((((λ#2332. ((#2332 (λ#2333. (λ#2334. (λ#2335. (((#2333 (λ#2336. (λ#2337. (#2337 (#2336 #2334))))) (λ#2338. #2335)) (λ#2339. #2339)))))) #2330)) #2331) (λ#2340. (λ#2341. (λ#2342. #2342)))) (λ#2343. (λ#2344. #2343))))) (λ#

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λ#3421. (λ#3422. (((λ#3423. (λ#3424. ((((λ#3425. (λ#3426. (λ#3427. ((#3425 #3426) #3427)))) #3423) #3424) #3423))) (((λ#3428. (λ#3429. ((λ#3430. ((#3430 (λ#3431. (λ#3432. (λ#3433. #3433)))) (λ#3434. (λ#3435. #3434)))) (((λ#3436. (λ#3437. ((#3437 (λ#3438. (λ#3439. (λ#3440. (((#3438 (λ#3441. (λ#3442. (#3442 (#3441 #3439))))) (λ#3443. #3440)) (λ#3444. #3444)))))) #3436))) #3428) #3429)))) #3421) #3422)) (((λ#3445. (λ#3446. ((λ#3447. ((#3447 (λ#3448. (λ#3449. (λ#3450. #3450)))) (λ#3451. (λ#3452. #3451)))) (((λ#3453. (λ#3454. ((#3454 (λ#3455. (λ#3456. (λ#3457. (((#3455 (λ#3458. (λ#3459. (#3459 (#3458 #3456))))) (λ#3460. #3457)) (λ#3461. #3461)))))) #3453))) #3445) #3446)))) #3422) #3421)))) (λ#3462. (λ#3463. (#3462 (#3462 #3463))))) (λ#3464. (λ#3465. (#3464 (#3464 #3465)))))
1: (((λ#3466. (λ#3467. (((λ#3468. (λ#3469. ((((λ#3470. (λ#3471. (λ#3472. ((#3470 #3471) #3472)))) #3468) #3469) #3468))) (((λ#3473. (λ#3474. ((λ#3475. ((#3475 (λ#3476. (λ#3477. (λ#3478. #3478)))) (λ#3479. (λ#3480.

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [20]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λ#26189. (λ#26190. (((λ#26191. (λ#26192. ((((λ#26193. (λ#26194. (λ#26195. ((#26193 #26194) #26195)))) #26191) #26192) #26191))) (((λ#26196. (λ#26197. ((λ#26198. ((#26198 (λ#26199. (λ#26200. (λ#26201. #26201)))) (λ#26202. (λ#26203. #26202)))) (((λ#26204. (λ#26205. ((#26205 (λ#26206. (λ#26207. (λ#26208. (((#26206 (λ#26209. (λ#26210. (#26210 (#26209 #26207))))) (λ#26211. #26208)) (λ#26212. #26212)))))) #26204))) #26196) #26197)))) #26189) #26190)) (((λ#26213. (λ#26214. ((λ#26215. ((#26215 (λ#26216. (λ#26217. (λ#26218. #26218)))) (λ#26219. (λ#26220. #26219)))) (((λ#26221. (λ#26222. ((#26222 (λ#26223. (λ#26224. (λ#26225. (((#26223 (λ#26226. (λ#26227. (#26227 (#26226 #26224))))) (λ#26228. #26225)) (λ#26229. #26229)))))) #26221))) #26213) #26214)))) #26190) #26189)))) (λ#26230. (λ#26231. (#26230 #26231)))) (λ#26232. (λ#26233. (#26232 (#26232 (#26232 (#26232 #26233)))))))
1: (((λ#26234. (λ#26235. (((λ#26236. (λ#26237. ((((λ#26238. (λ#26239. (λ#26240. ((#26238 #26239) #26240)))) #26236) #